In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
from torch.optim import AdamW
import numpy as np
import copy
from collections import deque
import gymnasium as gym
from gymnasium.spaces import Discrete, Box
from itertools import count
import random
from gymnasium.wrappers import NormalizeObservation, NormalizeReward
import torch.optim as optim

import matplotlib.pyplot as plt

In [2]:
#!pip install ipdb
#import ipdb

In [3]:
env_name = 'LunarLanderContinuous-v2'

In [4]:
# Actor Network
class ActorNet(nn.Module):
    def __init__(self, input_size, hidden_units=64, output_size=2):
        super(ActorNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_units),
            nn.Tanh(),
            nn.Linear(hidden_units, int(hidden_units/2)),
            nn.Tanh()
        )
        self.mu_head = nn.Linear(int(hidden_units/2),  output_size)
        self.logstd_head = nn.Linear(int(hidden_units/2),  output_size)

    def forward(self, x):
        x = self.model(x)
        loc = torch.tanh(self.mu_head(x)) * 2
        scale = torch.exp(self.logstd_head(x))
        return loc, scale

    def __call__(self, x):
        out = self.forward(x)
        return out

In [5]:
class CriticNet(nn.Module):
    def __init__(self, input_size, hidden_units=64, output_size=2):
        super(CriticNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_units),
            nn.Tanh(),
            nn.Linear(hidden_units, int(hidden_units/2)),
            nn.Tanh()
        )
        self.value_head = nn.Linear(int(hidden_units/2), 1)

    def forward(self, x):
        x = self.model(x)
        value = self.value_head(x)
        return value

    def __call__(self, x):
        out = self.forward(x)
        return out

In [6]:
def initialize_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data, nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight.data, 1)
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data)
        nn.init.constant_(m.bias.data, 0)

In [7]:
nenvs=64
env = gym.vector.make(env_name, num_envs=nenvs, asynchronous=False) #, new_step_api=True
#env = NormalizeObservation(env)
#env = NormalizeReward(env)
obs_dim = env.single_observation_space.shape[0]
n_acts = env.single_action_space.shape[0]

In [8]:
# Actor-Critic networks
hidden_sizes = 128
actor_net = ActorNet(obs_dim, hidden_sizes, n_acts)
actor_net = actor_net.apply(initialize_weights)

critic_net = CriticNet(obs_dim, hidden_sizes, n_acts)
critic_net = critic_net.apply(initialize_weights)

In [9]:
# optimizer
actor_optimizer = AdamW(actor_net.parameters(), lr=0.0003)
critic_optimizer = AdamW(critic_net.parameters(), lr=0.0003)

In [10]:
T = lambda x: torch.as_tensor(x, dtype=torch.float32)
Ti = lambda x: torch.as_tensor(x, dtype=torch.int64)

In [11]:
class RunningMem():

    def __init__(self):
        self.reset()

    def store(self, obs, action, logprob, reward, done, obs_, values, values_):
        self.obs.append(obs)
        self.actions.append(action.unsqueeze(-1))
        self.logprobs.append(logprob)
        self.rewards.append(reward.unsqueeze(-1))
        self.dones.append(done.unsqueeze(-1))
        self.obs_.append(obs_)
        self.values.append(values)
        self.values_.append(values_)


    def batches(self, batchsize):
        size = nenvs*memsteps
        idx = list(range(size))
        random.shuffle(idx)

        b_obs = torch.stack(self.obs)
        b_actions = torch.stack(self.actions)
        b_logprobs = torch.stack(self.logprobs)
        b_rewards = torch.stack(self.rewards)
        b_dones = torch.stack(self.dones)
        b_obs_ = torch.stack(self.obs_)
        b_values = torch.stack(self.values)
        b_values_ = torch.stack(self.values_)

        gaes = []
        gae = T(np.zeros(nenvs)).view(nenvs,-1)
        for i in range(len(b_obs)-1,-1,-1):
            delta = b_rewards[i] + gamma * b_values_[i] * (1-b_dones[i]) - b_values[i]
            gae = delta + gamma * lmbda * (1-b_dones[i]) * gae
            gaes.insert(0, gae)
        
        b_obs = b_obs.view(size, -1)
        b_actions = b_actions.view(size, -1)
        b_logprobs = b_logprobs.view(size, -1)
        b_rewards = b_rewards.view(size, -1)
        b_dones = b_dones.view(size, -1)
        b_obs_ = b_obs_.view(size, -1)
        b_values = b_values.view(size, -1)
        b_values_ = b_values_.view(size, -1)
        b_gae = torch.stack(gaes).view(size, -1)

        for batchn in range(0, len(idx), batchsize):
            batchidx = idx[batchn:batchn+batchsize]
            batchidx = Ti(batchidx)
            mb_obs = torch.index_select(b_obs, 0, batchidx)
            mb_actions = torch.index_select(b_actions, 0, batchidx)
            mb_logprobs = torch.index_select(b_logprobs, 0, batchidx)
            mb_rewards = torch.index_select(b_rewards, 0, batchidx)
            mb_dones = torch.index_select(b_dones, 0, batchidx)
            mb_obs_ = torch.index_select(b_obs_, 0, batchidx)
            mb_values = torch.index_select(b_values, 0, batchidx)
            mb_values_ = torch.index_select(b_values_, 0, batchidx)
            mb_gae = torch.index_select(b_gae, 0, batchidx)
            yield mb_obs, mb_actions, mb_logprobs, mb_rewards, mb_dones, mb_obs_, mb_values, mb_values_, mb_gae


    def reset(self):
        self.obs = []
        self.actions = []
        self.logprobs = []
        self.rewards = []
        self.dones = []    
        self.obs_ = []
        self.values = []
        self.values_ = []
        self.gae = []

In [12]:
@torch.no_grad()
def sim_action(policy, obs):
    loc, std = policy(T(obs))
    dist = Normal(loc=loc, scale=std+1e-6)
    action = dist.sample()
    action_log_prob = torch.sum(dist.log_prob(action), dim=-1, keepdim=True)
    return action, action_log_prob

In [13]:
def train(mem, gamma=0.99, batchsize=10, epoch_repeat=20, epsilon=0.2, lmbda=0.95):
    for epochrep in range(epoch_repeat):
        for batch in mem.batches(batchsize=batchsize):
            obs, actions, logprobs, rewards, dones, obs_, values, values_, gae = batch
            gae = (gae - torch.mean(gae)) / (torch.std(gae) + 1e-6)
            target = gae + values
            state_values = critic_net(obs)
            critic_loss = F.smooth_l1_loss(state_values, target).mean()
            
            new_loc, new_scale = actor_net(obs)
            dist = Normal(loc=new_loc, scale=new_scale+1e-6)
            new_logprobs = torch.sum(dist.log_prob(actions), dim=-1, keepdim=True)
            rho = torch.exp(new_logprobs - logprobs)
            surrgt1 = rho * gae
            surrgt2 = rho.clamp(1-epsilon, 1+epsilon) * gae
            policy_loss = -torch.minimum(surrgt1, surrgt2).mean()

            loss = policy_loss + 0.5*critic_loss
            actor_optimizer.zero_grad()
            critic_optimizer.zero_grad()
            loss.backward()
            actor_optimizer.step()
            critic_optimizer.step()


In [14]:
results = deque(maxlen=50)
graph_results = []
memsteps = 50
mem = RunningMem()
gamma=0.99
lmbda=0.95
epsilon = 0.18
batchsize=16
epoch_repeat=15


totreward = np.zeros(nenvs)
stepcount = 0
epoc = 0
obs,info = env.reset()

while True:
    stepcount+=1
    action, action_log_prob = sim_action(actor_net, obs)
    next_obs, reward, terminated, truncated, _ = env.step(action.numpy())
    with torch.no_grad():
            values = critic_net(T(obs))
            values_ = critic_net(T(next_obs))    
    mem.store(T(obs), action, action_log_prob, T(reward), Ti(terminated), T(next_obs), values, values_)
    obs = next_obs
    totreward += reward
    doneidx = np.where(terminated==True)
    for k in doneidx[0]:
        results.append(totreward[k])
        totreward[k] = 0
            
    if stepcount>1 and stepcount % memsteps == 0:
        epoc+=1
        train(mem, gamma=gamma, batchsize=batchsize, epoch_repeat=epoch_repeat, epsilon=epsilon, lmbda=lmbda)   
        mem.reset()
        graph_results.append(np.mean(results))
        if epoc % 10 == 0:
            print(f'Epoc: {epoc} Avg Result: {np.mean(results)}')   
    if len(results)>0 and np.mean(results) > 195:
        print(f'Solved!  Epoc: {epoc} Avg Result: {np.mean(results)}')
        break


c:\Users\mvell\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\mvell\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoc: 10 Avg Result: -486.89492170717966
Epoc: 20 Avg Result: -462.87235452719557
Epoc: 30 Avg Result: -438.12947609979375
Epoc: 40 Avg Result: -438.12947609979375
Epoc: 50 Avg Result: -414.4282306722894
Epoc: 60 Avg Result: -403.68455519898254
Epoc: 70 Avg Result: -384.3109122362325
Epoc: 80 Avg Result: -337.2075653658444
Epoc: 90 Avg Result: -334.10914857465696
Epoc: 100 Avg Result: -259.422525899078
Epoc: 110 Avg Result: -259.422525899078
Epoc: 120 Avg Result: -225.60209307856994
Epoc: 130 Avg Result: -212.33616784665776
Epoc: 140 Avg Result: -204.16435090960147
Epoc: 150 Avg Result: -196.75194373424765
Epoc: 160 Avg Result: -196.75194373424765
Epoc: 170 Avg Result: -196.75194373424765
Epoc: 180 Avg Result: -180.578261684798
Epoc: 190 Avg Result: -180.578261684798
Epoc: 200 Avg Result: -125.41469744201018
Epoc: 210 Avg Result: -61.160547894457686
Epoc: 220 Avg Result: -36.1967882937392
Epoc: 230 Avg Result: -28.942109823401598
Epoc: 240 Avg Result: -11.794712766588962
Epoc: 250 Avg 

In [1]:
x = range(1,epoc+1)
y = graph_results

plt.plot(x, y)
plt.xlabel('Epochs')
plt.ylabel('Results')
plt.show()

NameError: name 'epoc' is not defined